## Рекомендательные системы: товары, которые больше понравятся

Делаем рекомендации категорий товаров в интернет-магазине.

Для каждого пользователя известно, когда и по товару какой категории он кликал.


#### Метрика качества - mean average precision@k. 
Посмотрим детальнее на метрику: 
- precision@k - доля купленного из рекомендованного

- average precision@k - внутри одного пользователя (одной рекомендации) усредняем precision@k по позициям k (так решаем проблему предыдущей метрики - она не учитывала порядок элементов в "топе")

- mean average precision@k - усредняем по пользователям


In [1]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

**Задача:** хотим предсказывать, какая категория товара пользователю будет интересна на оснве информации о кликах. 
Т.е. построить модель, предсказываюшую вероятность клика.

**Общая схема построение модели:**

- обучение модели: объектом выборки будет являться пара (user_id, category_id). Таргет - "клик"/"не клик", т.е. бинарная классификация. Будем обучать бустинг на logloss, чтобы получить вероятность, по которой в дальнейшем ранжировать.

- здесь надо подумать о том, что мы будем считать "не кликом" (негативным классом), ведь у нас етсь только информация о кликах. Будем выбирать рандомно **несколько** негативных примеров из всех оставшихся категорий, по которые пользователь не кликнул). (**несколько**, а не все возможные, потому что в противном случае получится сильный дисбаланс классов)

- предсказание: для каждого пользователя из тестовой выборки и каждой возможной категории генерим признаки.

- принимаем решение согласно модели: выберем топ 5 категорий для каждого пользователя по полученной вероятности.


In [2]:
import numpy as np
import pandas as pd
from scipy import sparse
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

import random

In [3]:
clicks = pd.read_csv('clicks.csv', sep=',')


**1.** Разбейте данные на три части: 
- все до предпоследней недели (по этой неделе будем считать фичи о пользователе, о товаре, взаимные фичи)
- предпоследняя неделя (по этой неделе сгенерируем обучающую выборку) 
- последняя неделя (тест, на котором смотрим качество -- пользователи, для которых хотим предсказать. Замечание: разумеется, здесь есть некоторая "утечка": на практике возможно предсказывать для тех пользователей, которые 1) пришли до момента предсказания 2) обычно мы не знаем, кто из них появится в нашей системе на интересующей неделе). В этой задаче мы не будем обращать на это внимание.

In [4]:
pred_last_week_start = '2016-09-12' 
last_week_start = '2016-09-19' 

In [5]:
stat_clicks = clicks[clicks['day'] < pred_last_week_start]
train_clicks = clicks[(clicks['day'] < last_week_start)&(clicks['day'] >= pred_last_week_start)]
test_clicks = clicks[clicks['day'] >= last_week_start]

In [6]:
stat_clicks.head()

,user_id,category_id,day
0,46,672,2016-08-04
1,48,170,2016-08-04
2,48,170,2016-08-04
3,53,1190,2016-08-04
4,93,56,2016-08-04


**2.** По stat_clicks хотим посчитать статистики для фичей о пользователях и товарах.

- создайте в датафрейме столбец 'list', в котором будет лежать лист из "троек" (user_id, category_id, day)
- составим обучающую выборку (сами объекты, пока что без признакового описания): для каждого пользователя возьмите категории, которые входили в его топ-5 кликнутых (это будет класс 1), и еще 5 случайных категорий (класс 0).

Чем плох следующий вариант: для всех категорий, которые не просмотрел пользователь, поставить класс 0?

In [7]:
stat_clicks['list'] = stat_clicks[['user_id', 'category_id', 'day']].values.tolist()

C:\Users\panov\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
stat_clicks.head()

,user_id,category_id,day,list
0,46,672,2016-08-04,"[46, 672, 2016-08-04]"
1,48,170,2016-08-04,"[48, 170, 2016-08-04]"
2,48,170,2016-08-04,"[48, 170, 2016-08-04]"
3,53,1190,2016-08-04,"[53, 1190, 2016-08-04]"
4,93,56,2016-08-04,"[93, 56, 2016-08-04]"


In [9]:
stat_clicks = stat_clicks.groupby(by='user_id')['list'].apply(list) 

In [13]:
stat_clicks.groupby(by='user_id').head()

user_id
0                                    [[0, 672, 2016-08-25]]
1         [[1, 428, 2016-08-19], [1, 44, 2016-09-01], [1...
2                                    [[2, 892, 2016-08-11]]
3         [[3, 1257, 2016-08-06], [3, 2318, 2016-08-10],...
4                                    [[4, 108, 2016-08-10]]
5            [[5, 2149, 2016-08-12], [5, 2149, 2016-08-12]]
6                                    [[6, 977, 2016-08-15]]
7                                    [[7, 696, 2016-08-10]]
8         [[8, 134, 2016-08-06], [8, 134, 2016-08-17], [...
9                                    [[9, 672, 2016-08-11]]
11                                 [[11, 1407, 2016-08-31]]
13           [[13, 672, 2016-08-30], [13, 672, 2016-08-30]]
14                                  [[14, 672, 2016-09-06]]
15                                 [[15, 1819, 2016-08-28]]
17                                  [[17, 675, 2016-08-19]]
19                                  [[19, 429, 2016-09-02]]
20        [[20, 66, 2016-08-05],

In [15]:
stat_clicks = pd.DataFrame({'list':stat_clicks.values, 'userID':stat_clicks.index})
# для каждого пользователя сортируем по таймстэмпу
stat_clicks['list'] = stat_clicks['list'].apply(lambda x: sorted(x, key=lambda y: y[2]))

In [16]:
stat_clicks.loc[1].list

[[1, 428, '2016-08-19'],
 [1, 44, '2016-09-01'],
 [1, 44, '2016-09-01'],
 [1, 1967, '2016-09-01'],
 [1, 1967, '2016-09-01']]

In [20]:
categories = np.unique(clicks.category_id.values)

# функция возврашает лейблы 1/0 в y (здесь же сэмплируем негативные примеры)
# айди категории, айди пользователя в X 

def get_sample(train_clicks_list):
    sample, target = [], []

    for id, el in zip(train_clicks_list.index, train_clicks_list):
        _el = Counter(el).most_common(5)
        for cat in _el:
            sample.append([cat[0], id])
            target.append(1) 
        # в массив arr добавляем 5 рандомных элементов, которые не пересекаются с теми, которые уже были
        a = list(set.difference(set(random.sample(list(categories), 10)), set(_el)))
        arr = [[cat, id] for cat in a[:5]]
        sample.extend(arr)
        target.extend([0 for _ in range(5)])
    return target, sample
    
train_clicks_list = train_clicks.groupby('user_id')['category_id'].apply(list)   
targets, objects = get_sample(train_clicks_list)

In [26]:
train_clicks_list.head()

user_id
8     [103, 755, 755, 755, 429, 429, 429, 429]
16                             [672, 672, 672]
18                                       [951]
34    [771, 163, 163, 163, 163, 163, 163, 672]
44                                       [461]
Name: category_id, dtype: object

**3.**
Напишите код, который считает вспомогательные статистики по пользователям и категориям.


Признаки по категории:
- сколько всего кликов было по этой категории
- сколько уникальных пользователей кликало по этой категории
- сколько категорий встречалось с конкретной категорией (внутри одного пользователя) = топ 5 категорий, которые встречались вместе с конкретной внутри одного пользователя

По пользователю:
- для каждого пользователя список всех категорий, по которым он кликал (затем каждую из этих категорий будем кодировать числом - суммарным числом кликов по ней )
- посчитаем топ 5 самых популярных категорий (по всей выборке), затем посмотрим, кликал ли пользователь по этим 5 популярным (это будет 5 фичей для каждого пользователя = (просматривал ли категории из топ 5 когда-либо))

In [27]:
from collections import defaultdict, Counter

In [29]:
# по пользователю
categories_on_users = defaultdict(list)

for i in stat_clicks.index:
    clicks_on_users = stat_clicks.loc[i].list
    cats_on_users = [cat[1] for cat in clicks_on_users]
    categories_on_users[i].extend(cats_on_users)

# 5 самых поулярных категорий
popular_cats = [el[0] for el in Counter(clicks.category_id.values).most_common(5)]

# по категориям
# сколько всего кликов было по этой категории
clicks_on_category = Counter()
# сколько уникальных пользователей кликало по этой категории
users_on_category = defaultdict(set)
# сколько категорий встречалось с конкретной категорией (внутри одного пользователя)
category_with_category = defaultdict(list) 

for i in stat_clicks.index:
    clicks_on_users = stat_clicks.loc[i].list
    
    cats_on_users = set([cat[1] for cat in clicks_on_users])
    for cl in clicks_on_users:
        clicks_on_category[cl[1]] += 1
        category_with_category[cl[1]].extend(list(cats_on_users)) 
        users_on_category[cl[1]].add(cl[0])    
        
# топ 5 категорий, которые встречались с данной внутри одного пользователя
category_with_category_cnt = {}
for c in category_with_category:
    category_with_category_cnt[c] = [el[0] for el in Counter(category_with_category[c]).most_common(5)]
    
# число пользователей, которые хоть раз кликнули по категории
users_on_category_dict = {}
for k, v in users_on_category.items():
    users_on_category_dict[k] = len(v)

In [30]:
stat_clicks.head()

,list,userID
0,"[[0, 672, 2016-08-25]]",0
1,"[[1, 428, 2016-08-19], [1, 44, 2016-09-01], [1...",1
2,"[[2, 892, 2016-08-11]]",2
3,"[[3, 1257, 2016-08-06], [3, 2318, 2016-08-10],...",3
4,"[[4, 108, 2016-08-10]]",4


**4.** Для каждого объекта обучающей выборке (objects) создайте следующие признаки:
- сколько было кликов по этой категории
- сколько было различных пользователей, кликнувших по этой категори
- топ 5 которые встречались с данной категорией category_with_category_cnt: их кодируем встречаемостью всех кликов
- топ 5 самых крутых категорий по пользователю, закодированных числом кликов по ней со всей выборки
- 5 OHE-фичей (кликал ли пользователь 5 популярным)

Обучите xgboost.

In [51]:
X_train_feats = []

for cat_id, user_id in objects:
    
    features_train  = []
    # фичи по категории
    features_train.append(clicks_on_category[cat_id]) #сколько было кликов по этой категории
    features_train.append(users_on_category_dict.get(cat_id, 0)) #сколько было различных пользователей, кликнувших по этой категории
    #топ 5 которые встречались с данной категорией category_with_category_cnt: их кодируем встречаемостью всех кликов
    c_l = category_with_category_cnt.get(cat_id, popular_cats)
    if len(c_l) < 5:
        c_l = popular_cats
    features_train.extend([clicks_on_category.get(c_id, 0) for c_id in c_l])
    
    # фичи по пользователю
    # топ 5 самых крутых по пользователю
    all_cats = [el[0] for el in Counter(categories_on_users[user_id]).most_common(5)]
    i = 0
    for c in all_cats:
        # число пользователей, которые хоть раз кликнули по этой же категории
        features_train.append(users_on_category_dict[c])
        i+=1
    while i < 5:
        features_train.append(0)
        i+=1
        
    # есть ли у этого пользователя категории из топ 5
    for c in popular_cats:
        features_train.append(int(c in categories_on_users[user_id]))
        
    X_train_feats.append(features_train)
    
X_train_feats = np.array(X_train_feats)

In [53]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier().fit(X_train_feats, np.array(targets))

**5.** Теперь сделаем предсказание для каждого user_id  и каждой категории.
- создайте список всех категорий, для которой мы хотим предсказывать (внутри каждого пользователя)
- напишите функцию которая по user_id будет генерировать признаковое описание для каждой возможной категории

In [56]:
test_clicks_list = test_clicks.groupby('user_id')['category_id'].apply(list)

In [57]:
# все категории, для которой мы хотим предсказывать (внутри каждого пользователя)
categories_to_pred = np.unique(clicks.category_id.values)

In [58]:
def get_features(user_id, cat_id):

    features_train  = []
    # фичи по категории
    features_train.append(clicks_on_category[cat_id]) #сколько было кликов по этой категории
    features_train.append(users_on_category_dict.get(cat_id, 0)) #сколько было различных пользователей, кликнувших по этой категории
    #топ 5 которые встречались с данной категорией category_with_category_cnt: их кодируем встречаемостью всех кликов
    c_l = category_with_category_cnt.get(cat_id, popular_cats)
    if len(c_l) < 5:
        c_l = popular_cats
    features_train.extend([clicks_on_category.get(c_id, 0) for c_id in c_l])

    # фичи по пользователю
    # топ 5 самых крутых по пользователю
    all_cats = [el[0] for el in Counter(categories_on_users[user_id]).most_common(5)]
    i = 0
    for c in all_cats:
        # число пользователей, которые хоть раз кликнули по этой же категории
        features_train.append(users_on_category_dict[c])
        i+=1
    while i < 5:
        features_train.append(0)
        i+=1

    # есть ли у этого пользователя категории из топ 5
    for c in popular_cats:
        features_train.append(int(c in categories_on_users[user_id]))

    return features_train

In [59]:
def get_prediction(user_id):
    X_test_gen = np.array([get_features(user_id, c_id) for c_id in categories_to_pred])
    pred = gb.predict_proba(X_test_gen)[:, 1]
    ind = np.argpartition(pred, -5)[-5:] # индексы категорий
    return categories_to_pred[ind]

import heapq
# для оптимизации использовать heapq
# heapq.nlargest(5, zp(categories_to_pred, pred), key=1)

In [60]:
get_prediction(100)

array([120, 253, 440, 826, 672])

**6.** Смотрим качество:

In [65]:
m = []
for user_id, arr in zip(test_clicks_list.index, test_clicks_list)[1800:2000]:
    actual = test_clicks_list.loc[user_id]
    predicted = get_prediction(user_id)
    m.append(apk(actual, predicted, 5))

In [66]:
np.mean(m)

0.047325

Какие фичи еще можно исопльзовать?

- временные: день недели играет большую роль (в выходные и праздничные дни пользователи ведут себя не так, как в будние)
- признаки по пользователю и категории с учетом времени (просматривал ли пользователь эту категорию неделю назад, 2 недели назад и т.д.)